# Multiple models forecast h horizon
Por defecto se utiliza el mismo modelo para predecir un step a futuro y luego de forma recursiva se generan los features para predecir a h horizonte de tiempo a futuro.

Sin embargo, también es posible predecir a h horizonte de tiempo a futuro utilizando un modelo distinto para precedir un horizonte distinto. Por ejemplo h=3, se entrena un modelo que prediga t+1, otro que prediga t+2 y por último otro que prediga t+3.

Esto podría generar mejores resultados, pero toma mucho más tiempo de entrenamiento

SOURCE: https://nixtlaverse.nixtla.io/mlforecast/docs/how-to-guides/one_model_per_horizon.html

By default mlforecast uses the recursive strategy, i.e. a model is trained to predict the next value and if we’re predicting several values we do it one at a time and then use the model’s predictions as the new target, recompute the features and predict the next step.

There’s another approach where if we want to predict 10 steps ahead we train 10 different models, where each model is trained to predict the value at each specific step, i.e. one model predicts the next value, another one predicts the value two steps ahead and so on. This can be very time consuming but can also provide better results. If you want to use this approach you can specify max_horizon in MLForecast.fit, which will train that many models and each model will predict its corresponding horizon when you call MLForecast.predict.

In [1]:
import random
import lightgbm as lgb
import pandas as pd
from datasetsforecast.m4 import M4, M4Info
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import smape

from mlforecast import MLForecast
from mlforecast.lag_transforms import ExponentiallyWeightedMean, RollingMean
from mlforecast.target_transforms import Differences

### 0. Data

In [2]:
group = 'Hourly'
await M4.async_download('data', group=group)
df, *_ = M4.load(directory='data', group=group)
df['ds'] = df['ds'].astype('int')
ids = df['unique_id'].unique()
random.seed(0)
sample_ids = random.choices(ids, k=4)
sample_df = df[df['unique_id'].isin(sample_ids)]
info = M4Info[group]
horizon = info.horizon
valid = sample_df.groupby('unique_id').tail(horizon)
train = sample_df.drop(valid.index)

In [3]:
def avg_smape(df):
    """Computes the SMAPE by serie and then averages it across all series."""
    full = df.merge(valid)
    return (
        evaluate(full, metrics=[smape])
        .drop(columns='metric')
        .set_index('unique_id')
        .squeeze()
    )

In [4]:
horizon

48

### 1. Train

In [7]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        1: [RollingMean(window_size=24)],
        24: [RollingMean(window_size=24)],
        48: [ExponentiallyWeightedMean(alpha=0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24])],
)

In [8]:
fcst

MLForecast(models=[LGBMRegressor], freq=1, lag_features=['lag24', 'lag48', 'lag72', 'lag96', 'lag120', 'lag144', 'lag168', 'rolling_mean_lag1_window_size24', 'rolling_mean_lag24_window_size24', 'exponentially_weighted_mean_lag48_alpha0.3'], date_features=[], num_threads=1)

In [9]:
horizon = 24

In [10]:
# the following will train 24 models, one for each horizon
individual_fcst = fcst.fit(train, max_horizon=horizon) #### AL DEFINIR EL PARAM MAX_HORIZON EN EL ENTRENAMIENTO SE ENTRENA UN MODELO POR STEP
individual_preds = individual_fcst.predict(horizon)
avg_smape_individual = avg_smape(individual_preds).rename('individual')

In [11]:
# the following will train a single model and use the recursive strategy
recursive_fcst = fcst.fit(train)
recursive_preds = recursive_fcst.predict(horizon)
avg_smape_recursive = avg_smape(recursive_preds).rename('recursive')

In [12]:
# results
print('Average SMAPE per method and serie')
avg_smape_individual.to_frame().join(avg_smape_recursive).applymap('{:.1%}'.format)

Average SMAPE per method and serie


,individual,recursive
unique_id,,
H196,0.3%,0.3%
H256,0.4%,0.3%
H381,20.9%,9.5%
H413,11.9%,13.6%


### En el output se puede observar, que no necesariamente es mejor entrenar un modelo por step